# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\dev\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)
train['Mensagem']=train['Mensagem'].astype('category')

In [4]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,"Target (Neutro, Positivo, Negativo)"
0,"Apesar da narrativa prender o leitor, os argum...",negativo
1,A formatação desse ebook atrapalha muito a lei...,neutro
2,Não é o primeiro livro da Record que oxida em ...,neutro
3,O hype que envolve esse livro é ENORME então e...,negativo
4,"meu deus, essa escritora conseguiu estragar um...",negativo


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
def cleanup(text):
    text=text.lower()
    punctuation = '[´"!-.:?;$''<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remover_acentos(texto):
    # Use unidecode para remover acentos e converter caracteres especiais para caracteres ASCII
    texto_sem_acentos = unidecode(texto)
    return texto_sem_acentos


In [6]:
def df0(df):
    df['Mensagem'] = df['Mensagem'].apply(cleanup)
    df['Mensagem'] = df['Mensagem'].apply(remover_acentos)
    novo_df = pd.concat([df[['Mensagem']], df[['Target (Loja e Sistema, Entrega,Conteudo)']]], axis=1)
    return novo_df
train=df0(train)
train['Mensagem']=train['Mensagem'].astype('category')

In [7]:
df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
quant_loja=len(df_Loja_Sistema)
quant_Conteudo=len(df_Conteudo)
quant_Entrega=len(df_Entrega)

df_Loja_Sistema

,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)"
0,nao li ate pq nao consegui baixar vcs podem aj...,Loja e Sistema
1,nao tem nada indicando que e em portugues de p...,Loja e Sistema
8,nao consigo ler esta muito confuso este site ...,Loja e Sistema
12,comprei o livro mas ele nao vai para a bibliot...,Loja e Sistema
15,pelo amor de deus que preco e esteso mesmo no ...,Loja e Sistema
...,...,...
733,comprei o livro e ele nao baixou gostaria de s...,Loja e Sistema
739,ta valendo obrigado grande abraco e obrigado a...,Loja e Sistema
740,sem imaginacao inducao ao alcoolismo as drogas...,Loja e Sistema
745,livro bastante repetitivo onde o autor se prom...,Loja e Sistema


In [8]:

tabela_Loja_Sistema=df_Loja_Sistema.value_counts()
print(tabela_Loja_Sistema)
tabela_Conteudo=df_Conteudo.value_counts()
tabela_Entrega=df_Entrega.value_counts()
def quantidade_de_palavras(df):
    lista=[]
    for i in df["Mensagem"]:
        e=i.split()
        for u in e:
            lista.append(u)
    quantidade=len(lista)
    return quantidade
qtd_plv_Entrega=quantidade_de_palavras(df_Entrega)
qtd_plv_LojaeSistema=quantidade_de_palavras(df_Loja_Sistema)
qtd_plv_Conteudo=quantidade_de_palavras(df_Conteudo)
qtd_total_plv=qtd_plv_Entrega+qtd_plv_Conteudo+qtd_plv_LojaeSistema
prob_entrega=qtd_plv_Entrega/qtd_total_plv
prob_LojaeSistema=qtd_plv_LojaeSistema/qtd_total_plv
prob_conteudo=qtd_plv_Conteudo/qtd_total_plv
print("A probabilidade de uma palavra ser de um comentário relacionado à entrega: {0}".format(prob_entrega))
print("A probabilidade de uma palavra ser de um comentário relacionado à loja e sistema: {0}".format(prob_LojaeSistema))
print("A probabilidade de uma palavra ser de um comentário relacionado ao conteúdo do livro: {0}".format(prob_conteudo))

Mensagem                                                                                                                                                                                                                                                                                                                                                                    Target (Loja e Sistema, Entrega,Conteudo)
a edicao traz uma introducao que e puro spoiler muitos leitores estao reclamando da mesma coisa aqui por que nao colocam essa maldita analise no final                                                                                                                                                                                                                      Loja e Sistema                               2
acabo de ver a trilogia impressa por 5490 amazon esta sendo ridicula com esse preco pensava seriamente em comprar a edicao kindle mas vou ficar com os que tenho mesmo inviavel        

In [9]:
def laplace(q):
    return (1)/(q+qtd_total_plv)

In [11]:
frase=str(input('frase:'))
frase=cleanup(frase)
frase=remover_acentos(frase)
frase=frase.lower()
palavras=frase.split(' ')
prob_Loja_Sistema=1
prob_Entrega=1
prob_Conteudo=1
for palavra in palavras:
    if palavra in tabela_Loja_Sistema:
        prob_Loja_Sistema*=(tabela_Loja_Sistema[palavra]/qtd_plv_LojaeSistema)
    else:
        prob_Loja_Sistema*=laplace(qtd_plv_LojaeSistema)
for palavra in palavras:
    if palavra in tabela_Entrega:
        prob_Entrega*=(tabela_Entrega[palavra]/qtd_plv_Entrega)
    else:
        prob_Entrega*=laplace(qtd_plv_Entrega)
for palavra in palavras:
    if palavra in tabela_Conteudo:
        prob_Conteudo*=(tabela_Conteudo[palavra]/qtd_plv_Conteudo)
    else:
        prob_Conteudo*=laplace(qtd_plv_Conteudo)
maior_prob=max([prob_Conteudo,prob_Entrega,prob_Loja_Sistema])
if maior_prob==prob_Conteudo:
    print("Conteúdo do Livro")
elif maior_prob==prob_Entrega:
    print("Entrega")
else:
    print("Loja e Sistema")

2.3852685812422478e-05
1.67372420372571e-05
2.3544369364066585e-05


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**